#### load the dataset


In [39]:
### path = data/dataset
from PIL import Image
import os

### method to convert img to arr

In [40]:
import cv2
import numpy as np
from PIL import Image
import random

def modify_image(img):
    # Convert to PIL Image for easier rotation
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # 1. Random angle rotation
    angle = random.uniform(-30, 30)
    rotated = pil_img.rotate(angle, expand=True, fillcolor=(255, 255, 255))
    
    # 2. Off-center shift
    shift_x = random.randint(-20, 20)
    shift_y = random.randint(-20, 20)
    
    # Create a new image with a white background
    new_img = Image.new('RGB', (rotated.width + abs(shift_x), rotated.height + abs(shift_y)), (255, 255, 255))
    
    # Paste the rotated image onto the new image with the shift
    new_img.paste(rotated, (max(shift_x, 0), max(shift_y, 0)))
    
    # Convert back to OpenCV format
    cv_img = cv2.cvtColor(np.array(new_img), cv2.COLOR_RGB2BGR)
    
    # 3. Add noise
    # noise = np.random.normal(0, 10, cv_img.shape).astype(np.uint8)
    # noisy_img = cv2.add(cv_img, noise)
    
    return cv_img

# Example usage:
img = Image.open('../data/dataset/TRAIN/5/5_5.png')
modified_img = modify_image(np.array(img))
#show image
cv2.imshow('image', modified_img)
cv2.waitKey(0)
cv2.destroyAllWindows


<function destroyAllWindows>

In [41]:
import numpy as np
import cv2

def convert_img_to_binary_array(img):
    #convert image to rgba
    img = img.resize((40, 40))  
    img = img.convert('RGBA')

    #get the pixel values of the image
    pixels = list(img.getdata())
    binary_array = []
    for pixel in pixels:
        if pixel[0] >= 128 and pixel[1] >= 128 and pixel[2] >= 128:
            binary_array.append(1)
        else:
            binary_array.append(0)
    return binary_array

def add_image_to_X_train(image, X_train):
    # image = modify_image(image)
    # print(image.shape)

    # Resize the image to 40x40 if it's not already
    if image.shape[:2] != (40, 40):
        image = cv2.resize(image, (40, 40), interpolation=cv2.INTER_AREA)
    
    # Convert to grayscale if the image has more than one channel
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Convert to black and white (0 or 255)
    _, bw_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    
    # Normalize to 0 or 1 for the neural network
    normalized_image = (bw_image / 255).astype(np.float32)
    
    # Reshape to add channel dimension
    reshaped_image = normalized_image.reshape(1, 40, 40, 1)

    # Add to X_train
    if X_train is None:
        X_train = reshaped_image
    else:
        X_train = np.vstack((X_train, reshaped_image))
    return X_train

### create training and testing arrays

In [42]:
X_train = None
y_train = []

path = "../data/dataset/TRAIN/"
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if not os.path.isdir(folder_path):
        continue
    if folder == "27":
        continue
    print(f"Processing folder: {folder}")
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)

                X_train = add_image_to_X_train(np.array(img), X_train)

                y_train.append(int(folder))
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

print(X_train)
print(len(y_train))

Processing folder: 0
Processing folder: 1
Processing folder: 10
Processing folder: 11
Processing folder: 12
Processing folder: 13
Processing folder: 14
Processing folder: 15
Processing folder: 16
Processing folder: 17
Processing folder: 18
Processing folder: 19
Processing folder: 2
Processing folder: 20
Processing folder: 21
Processing folder: 22
Processing folder: 23
Processing folder: 24
Processing folder: 25
Processing folder: 26
Processing folder: 3
Processing folder: 4
Processing folder: 5
Processing folder: 6
Processing folder: 7
Processing folder: 8
Processing folder: 9
[[[[1.]
   [1.]
   [1.]
   ...
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   ...
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   ...
   [1.]
   [1.]
   [1.]]

  ...

  [[1.]
   [1.]
   [1.]
   ...
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   ...
   [1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]
   ...
   [1.]
   [1.]
   [1.]]]


 [[[1.]
   [1.]
   [1.]
   ...
   [1.]
   [1.]
   [1.]]

  [[1.]


In [43]:
X_test = None
y_test = []
path = "../data/dataset/TEST/"
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if not os.path.isdir(folder_path):
        continue
    if folder == "27":
        continue
    print(f"Processing folder: {folder}")
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)
                # binary_array = convert_img_to_binary_array(img)
                X_test = add_image_to_X_train(np.array(img), X_test)

                y_test.append(int(folder))
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

Processing folder: 0
Processing folder: 1
Processing folder: 10
Processing folder: 11
Processing folder: 12
Processing folder: 13
Processing folder: 14
Processing folder: 15
Processing folder: 16
Processing folder: 17
Processing folder: 18
Processing folder: 19
Processing folder: 2
Processing folder: 20
Processing folder: 21
Processing folder: 22
Processing folder: 23
Processing folder: 24
Processing folder: 25
Processing folder: 26
Processing folder: 3
Processing folder: 4
Processing folder: 5
Processing folder: 6
Processing folder: 7
Processing folder: 8
Processing folder: 9


In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## split data randomly

In [45]:
X = np.concatenate((X_train, X_test))
y = np.array(np.concatenate((y_train, y_test)))

print(X.shape[0])
print(len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

print(f"Train size: {X_train.shape}")
print(f"Test size: {len(y_train)}")

5073
5073
Train size: 4058
Test size: 1015
Train size: (4058, 40, 40, 1)
Test size: 4058


In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(40, 40, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(27, activation='softmax')  # 27 for 22 regular + 5 final forms
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\nizan\Desktop\letterRec\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
# convert to one-hot encoding
from tensorflow.keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train, 27)

history = model.fit(X_train, y_train_one_hot, 
                    epochs=20, 
                    batch_size=32)

Epoch 1/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0744 - loss: 3.1965
Epoch 2/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3248 - loss: 2.3169
Epoch 3/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4654 - loss: 1.8043
Epoch 4/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5326 - loss: 1.5567
Epoch 5/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5797 - loss: 1.4340
Epoch 6/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6121 - loss: 1.2863
Epoch 7/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6350 - loss: 1.2360
Epoch 8/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6685 - loss: 1.0992
Epoch 9/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6675 - loss: 1.0957
Epoch 10/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6921 - loss: 1.0198
Epoch 11/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6949 - loss: 0.9911
Epoch 12/20
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step

In [48]:
y_test_one_hot = to_categorical(y_test, 27)

test_loss, test_acc = model.evaluate(X_test, y_test_one_hot)
print(f'Test accuracy: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7559 - loss: 1.0214
Test accuracy: 0.7694581151008606


In [49]:
# save the model
model.save('../models/keras_model.keras')

In [50]:
# prediction
predictions = model.predict(X_test)
first_prediction = predictions[0]
actual = y_test[0]

print(f"First prediction: {first_prediction}")
print(f"Actual: {actual}")

# convert to DataFrame and show the y_test and if correct
predictions_dict = []
for i in range(len(predictions)):
    prediction = predictions[i]
    prediction_dict = {}
    for j in range(len(prediction)):
        prediction_dict[j] = prediction[j]
    predictions_dict.append(prediction_dict)

predictions_df = pd.DataFrame(predictions_dict)
predictions_df['actual'] = y_test
predictions_df['predicted'] = predictions
predictions_df['probability'] = predictions_df.apply(lambda x: x[x['predicted']], axis=1)
predictions_df['actual_probability'] = predictions_df.apply(lambda x: x[x['actual']], axis=1)
predictions_df['correct'] = predictions_df['actual'] == predictions_df['predicted']

print(predictions_df.head(10))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
First prediction: [8.2747126e-04 1.7023218e-03 9.7002060e-04 1.8449744e-02 4.9260709e-02
 4.6229060e-04 2.9227018e-01 1.0048589e-02 3.5537310e-02 3.7968715e-03
 9.3293056e-02 1.2245016e-02 2.6188243e-02 1.6550037e-04 3.9607254e-03
 8.3831772e-03 1.3979244e-03 2.9636942e-02 1.4408998e-03 8.8915852e-04
 8.3066285e-02 2.5245691e-02 2.9384827e-03 6.1563606e-04 1.6039240e-01
 1.6071467e-02 1.2074392e-01]
Actual: 19


ValueError: Expected a 1D array, got an array with shape (1015, 27)

## Create a KNN classifier

In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_predictions = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)

## Create a Decision Tree classifier

In [ ]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)

## Create a Random Forest classifier

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)

## Create a Naive Bayes classifier

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)

# Compare the accuracies of the models


In [ ]:
print("SVM Accuracy:", svm_accuracy)
# print("KNN Accuracy:", knn_accuracy)
# print("Decision Tree Accuracy:", dt_accuracy)
# print("Random Forest Accuracy:", rf_accuracy)
# print("Naive Bayes Accuracy:", nb_accuracy)

SVM Accuracy: 0.7497536945812808


In [ ]:
svm_accuracy = accuracy_score(y_test, svm_predictions)
knn_accuracy = accuracy_score(y_test, knn_predictions)
dt_accuracy = accuracy_score(y_test, dt_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)
nb_accuracy = accuracy_score(y_test, nb_predictions)

svm_precision = precision_score(y_test, svm_predictions, average='weighted')
knn_precision = precision_score(y_test, knn_predictions, average='weighted')
dt_precision = precision_score(y_test, dt_predictions, average='weighted')
rf_precision = precision_score(y_test, rf_predictions, average='weighted')
nb_precision = precision_score(y_test, nb_predictions, average='weighted')

svm_recall = recall_score(y_test, svm_predictions, average='weighted')
knn_recall = recall_score(y_test, knn_predictions, average='weighted')
dt_recall = recall_score(y_test, dt_predictions, average='weighted')
rf_recall = recall_score(y_test, rf_predictions, average='weighted')
nb_recall = recall_score(y_test, nb_predictions, average='weighted')

svm_f1 = f1_score(y_test, svm_predictions, average='weighted')
knn_f1 = f1_score(y_test, knn_predictions, average='weighted')
dt_f1 = f1_score(y_test, dt_predictions, average='weighted')
rf_f1 = f1_score(y_test, rf_predictions, average='weighted')
nb_f1 = f1_score(y_test, nb_predictions, average='weighted')

performance_metrics = pd.DataFrame({
    'Algorithm': ['SVM', 'KNN', 'Decision Tree', 'Random Forest', 'Naive Bayes'],
    'Accuracy': [svm_accuracy, knn_accuracy, dt_accuracy, rf_accuracy, nb_accuracy],
    'Precision': [svm_precision, knn_precision, dt_precision, rf_precision, nb_precision],
    'Recall': [svm_recall, knn_recall, dt_recall, rf_recall, nb_recall],
    'F1-Score': [svm_f1, knn_f1, dt_f1, rf_f1, nb_f1]
})

print(performance_metrics)

       Algorithm  Accuracy  Precision    Recall  F1-Score
0            SVM  0.749754   0.760278  0.749754  0.748954
1            KNN  0.566502   0.613491  0.566502  0.556145
2  Decision Tree  0.414778   0.415872  0.414778  0.411652
3  Random Forest  0.689655   0.696677  0.689655  0.684458
4    Naive Bayes  0.348768   0.388866  0.348768  0.310005


c:\Users\user\Desktop\נתניה\HebrewHandwrittenRecognition\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Test on a manual set

In [ ]:
X_manual = []
y_manual = []

path = "../data/dataset/manualTest/"
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if not os.path.isdir(folder_path):
        continue
    if folder == "27":
        continue
    print(f"Processing folder: {folder}")
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)
                binary_array = convert_img_to_binary_array(img)
                X_manual.append(binary_array)
                y_manual.append(int(folder))
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

Processing folder: 0
Processing folder: 1
Processing folder: 2
Processing folder: 3
Processing folder: 4


In [ ]:
#show manuak test accuracy
import pickle
with open('./svm_model.sav', 'rb') as file:
    svm_model = pickle.load(file)
manual_predictions = svm_model.predict(X_manual)
manual_accuracy = accuracy_score(y_manual, manual_predictions)
print("Manual Test Accuracy:", manual_accuracy)
print("Manual Test Predictions:", manual_predictions)
print("Manual Test Actual:", y_manual)

#predict the 4th image
img = Image.open("../data/dataset/manualTest/1/1_01.png")
img.show()
binary_array = convert_img_to_binary_array(img)
prediction = svm_model.predict([binary_array])[0]
print("Prediction:", prediction)

NameError: name 'X_manual' is not defined